In [ ]:
import rasterio
import rasterio.mask
import numpy as np
import shapely
import geopandas as gpd
import random
import shapely.geometry as sg
from pathlib import Path
from scipy import misc, ndimage
import cv2

import matplotlib.pyplot as plt

In [ ]:
data_dir = Path("data/raw")
img = rasterio.open(Path(data_dir) / "GL085796E28265N.tif")
x = img.read()
y = gpd.read_file(data_dir / "GL_3basins_2015.shp")
extent = gpd.GeoDataFrame(index=[0], crs=y.crs, geometry=[sg.box(*img.bounds)])
y_extent = gpd.overlay(y, extent)

In [ ]:
import data as dt

masks, p = [], []
for geom in y_extent["geometry"]:
    mask_i = rasterio.mask.mask(img, [geom])
    masks.append(1 * np.any(mask_i[0] != 0, axis=0))
    p.append(dt.extreme_points(masks[-1]))

In [ ]:
plt.imshow(np.transpose(x, (1, 2, 0)))
for pi in p:
    plt.scatter(pi[:, 0], pi[:, 1], s=3, c="red")
plt.show()

In [ ]:
p_blur = dt.make_gt(x.shape[1:], np.vstack(p))
plt.imshow(np.sum(p_blur, 0))

In [ ]:
dts, sdts = dt.sdt(np.stack(masks))
plt.imshow(np.sum(masks, axis=0))
plt.show()
plt.imshow(np.sum(sdts, axis=0))

In [ ]:
np.save(data_dir / "sdts.npy", sdts)
np.save(data_dir / "x.npy", x)
np.save(data_dir / "y.npy", np.stack(masks))